In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from collections import defaultdict
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import torch
from dysts.metrics import compute_metrics
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.io import loadmat
from tqdm import tqdm

from dystformer.chronos.pipeline import ChronosPipeline
from dystformer.patchtst.pipeline import PatchTSTPipeline
from dystformer.utils import safe_standardize

In [ ]:
if os.path.exists("../custom_style.mplstyle"):
    plt.style.use(["ggplot", "../custom_style.mplstyle"])

## Load Model Checkpoints

In [ ]:
run_name = "pft_chattn_emb_w_poly-0"
# run_name = "pft_chattn_noembed_pretrained_correct-0"  # chattn + mlm
# run_name = "pft_linattnpolyemb_from_scratch-0"

pft_model = PatchTSTPipeline.from_pretrained(
    mode="predict",
    pretrain_path=f"/stor/work/AMDG_Gilpin_Summer2024/checkpoints/{run_name}/checkpoint-final",
    device_map="cuda:2",
)

In [ ]:
chronos_ft = ChronosPipeline.from_pretrained(
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_finetune_stand_updated-0/checkpoint-final",
    # "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_mini_ft-0/checkpoint-final",
    "/stor/work/AMDG_Gilpin_Summer2024/checkpoints/chronos_bolt_mini-12/checkpoint-final",
    device_map="cuda:3",
    torch_dtype=torch.float32,
)

In [ ]:
chronos_zs = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-mini",
    device_map="cuda:4",
    torch_dtype=torch.float32,
)

## Forecast and Plot Utils

In [ ]:
def forecast(
    model,
    context: np.ndarray,
    prediction_length: int,
    transpose: bool = False,
    standardize: bool = True,
    differenced: bool = False,
    **kwargs,
) -> np.ndarray:
    """
    Args:
        model: The model to use for forecasting.
        context: The context to forecast (n_timesteps, n_features)
        context_length: The length of the context.
        prediction_length: The length of the prediction.
        transpose: Whether to transpose the data.

    Returns:
        The forecasted data (prediction_length, n_features)
    """
    preprocessed_context = context.copy()

    if differenced:
        differenced_context = np.diff(preprocessed_context, axis=0)
        preprocessed_context = differenced_context.copy()
    if standardize:
        preprocessed_context = safe_standardize(preprocessed_context, axis=0)

    context_tensor = torch.from_numpy(
        preprocessed_context.T if transpose else preprocessed_context
    ).float()
    pred = (
        model.predict(context_tensor, prediction_length, verbose=False, **kwargs)
        .squeeze()
        .cpu()
        .numpy()
    )
    if transpose:
        pred = pred.T

    if standardize:
        pred = safe_standardize(
            pred,
            axis=0,
            context=differenced_context if differenced else context,
            denormalize=True,
        )
    if differenced:
        pred = np.cumsum(pred, axis=0) + context[-1]

    # prediction length may be shorter than model output length
    return pred[:prediction_length, :]


def compute_rollout_metrics(
    model,
    data: np.ndarray,
    context_length: int,
    prediction_length: int,
    step: int = 64,
    num_windows: int = 5,
    metrics: list[str] = ["mse", "mae", "smape"],
    **kwargs,
) -> tuple[dict[str, np.ndarray], dict[str, np.ndarray], np.ndarray, list[np.ndarray]]:
    full_metrics = defaultdict(
        lambda: np.zeros((num_windows, prediction_length // step))
    )
    starts = np.random.randint(
        0, len(data) - context_length - prediction_length, num_windows
    )
    predictions = []
    for s in tqdm(range(num_windows), desc="Sampling contexts", total=num_windows):
        start = starts[s]
        context = data[start : start + context_length]
        prediction = forecast(model, context, prediction_length, **kwargs)
        for i in range(0, prediction_length, step):
            pred = prediction[i : i + step]

            gt = data[start + context_length + i : start + context_length + i + step]
            submetrics = compute_metrics(pred, gt, include=metrics)
            for k, v in submetrics.items():
                full_metrics[k][s, i // step] += v
        predictions.append(prediction)
    mean_metrics = {k: v.mean(axis=0) for k, v in full_metrics.items()}
    std_metrics = {
        k: v.std(axis=0) / np.sqrt(num_windows) for k, v in full_metrics.items()
    }
    return mean_metrics, std_metrics, starts, predictions


def plot_model_prediction(
    model,
    data: np.ndarray,
    context_length: int,
    prediction_length: int,
    transpose: bool = False,
    standardize: bool = True,
    save_path: str | None = None,
    color: str = "red",
    **kwargs,
):
    context = data[:context_length]
    groundtruth = data[context_length : context_length + prediction_length]
    prediction = forecast(
        model, context, prediction_length, transpose, standardize, **kwargs
    )

    total_length = context_length + prediction_length
    context_ts = np.arange(context_length + 1)
    pred_ts = np.arange(context_length, total_length)

    fig = plt.figure(figsize=(15, 4))
    outer_grid = fig.add_gridspec(1, 2, width_ratios=[0.5, 0.5], wspace=0.05)
    gs = outer_grid[1].subgridspec(3, 1, height_ratios=[1 / 3] * 3, wspace=0, hspace=0)
    ax_3d = fig.add_subplot(outer_grid[0], projection="3d")
    ax_3d.plot(*context.T[:3], alpha=0.5, color="black", label="Context")
    ax_3d.plot(*groundtruth.T[:3], linestyle="--", color="black", label="Groundtruth")
    ax_3d.plot(*prediction.T[:3], color=color, label="Prediction")
    ax_3d.legend(loc="upper right", fontsize=8)
    ax_3d.set_xlabel("$x_1$")
    ax_3d.set_ylabel("$x_2$")
    ax_3d.set_zlabel("$x_3$")

    axes_1d = [fig.add_subplot(gs[i, 0]) for i in range(3)]
    for i, ax in enumerate(axes_1d):
        ax.plot(context_ts, data[: context_length + 1, i], alpha=0.5, color="black")
        ax.plot(pred_ts, groundtruth[:, i], linestyle="--", color="black")
        ax.plot(pred_ts, prediction[:, i], color=color)
        ax.set_ylabel(f"$x_{i + 1}$")
        ax.set_aspect("auto")
    axes_1d[-1].set_xlabel("Time")

    if save_path is None:
        plt.show()
    else:
        plt.savefig(save_path)
    plt.close()

In [ ]:
def plot_comparison(
    data: np.ndarray,
    predictions_dict: dict[str, np.ndarray],
    context_length: int,
    prediction_length: int,
    figsize: tuple[int, int] = (6, 6),
    legend_kwargs: dict[str, Any] = {},
    save_path: str | None = None,
):
    context = data[: context_length + 1, :3]
    groundtruth = data[context_length : context_length + prediction_length, :3]

    plt.figure(figsize=figsize)
    ax = plt.axes(projection="3d")
    ax._axis3don = False

    # Combine all data to find min/max bounds
    all_data = [context, groundtruth] + [
        pred[:, :3] for pred in predictions_dict.values()
    ]
    mins = np.array([d.min(axis=0) for d in all_data])
    maxs = np.array([d.max(axis=0) for d in all_data])

    xmin, ymin, zmin = np.min(mins, axis=0)
    xmax, ymax, zmax = np.max(maxs, axis=0)

    ax.xaxis.pane.set_visible(False)
    ax.yaxis.pane.set_visible(False)
    ax.zaxis.pane.set_visible(False)
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_zticks([])

    ax.plot3D(*context.T, alpha=0.1, color="black", zorder=1)
    ax.plot3D(
        *groundtruth.T,
        alpha=0.8,
        color="black",
        linestyle="-",
        zorder=2,
        label="Ground Truth",
    )
    for model_name, prediction in predictions_dict.items():
        ax.plot3D(
            *prediction[:, :3].T,
            label=model_name,
            zorder=10 if model_name == "Our Model" else 1,
        )
    ax.legend(**legend_kwargs)

    ax.quiver(
        xmin,
        ymax,
        zmin,
        xmax - xmin,
        0,
        0,
        color="black",
        linewidth=2,
        arrow_length_ratio=0.05,
        zorder=5,
    )
    ax.quiver(
        xmin,
        ymax,
        zmin,
        0,
        -ymax + ymin,
        0,
        color="black",
        linewidth=2,
        arrow_length_ratio=0.05,
        zorder=5,
    )
    ax.quiver(
        xmin,
        ymax,
        zmin,
        0,
        0,
        zmax - zmin,
        color="black",
        linewidth=2,
        arrow_length_ratio=0.05,
        zorder=5,
    )

    plt.tight_layout()
    if save_path is not None:
        plt.savefig(save_path, bbox_inches="tight")
    plt.show()

# Double Pendulum

In [ ]:
SPLIT = "train"
INDEX = 0
WORK = os.environ.get("WORK", "")
base_dir = f"{WORK}/physics-datasets"
fpath = f"{base_dir}/double_pendulum_chaotic/train_and_test_split/dpc_dataset_traintest_4_200_csv/{SPLIT}/{INDEX}.csv"
pendulum_data = np.loadtxt(fpath)
print(pendulum_data.shape)

# data is non-stationary, subsample and detrend it
subsampled_pendulum_data = pendulum_data[::10, -4:]

## The position of the pivot point (mostly constant)
plt.plot(pendulum_data[:, 1], -pendulum_data[:, 0])

## The position of the tip of the first pendulum
plt.plot(pendulum_data[:, 3], -pendulum_data[:, 2])

## The position of the tip of the second pendulum
plt.plot(pendulum_data[:, 5], -pendulum_data[:, 4])

In [ ]:
context_length = 512
prediction_length = 128

metrics = ["mse", "mae", "smape"]

In [ ]:
pft_prediction = forecast(
    pft_model,
    subsampled_pendulum_data[:context_length],
    prediction_length,
    limit_prediction_length=False,
    sliding_context=True,
)


pft_metrics = defaultdict(list)
step = 8
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        pft_prediction[0 : i + step],
        subsampled_pendulum_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        pft_metrics[metric].append(submetrics[metric])
print(pft_metrics)

In [ ]:
chronos_prediction = forecast(
    chronos_ft,
    subsampled_pendulum_data[:context_length],
    prediction_length,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    deterministic=True,
)

chronos_metrics = defaultdict(list)
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        chronos_prediction[0 : i + step],
        subsampled_pendulum_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        chronos_metrics[metric].append(submetrics[metric])
print(chronos_metrics)

In [ ]:
plt.figure(figsize=(10, 10))

## The position of the tip of the second pendulum
plt.plot(
    subsampled_pendulum_data[:context_length, 3],
    -subsampled_pendulum_data[:context_length, 2],
    alpha=0.1,
    color="black",
)
plt.plot(
    subsampled_pendulum_data[context_length : context_length + prediction_length, 3],
    -subsampled_pendulum_data[context_length : context_length + prediction_length, 2],
    alpha=0.5,
    color="black",
    linestyle="--",
)
plt.plot(pft_prediction[:, 3], -pft_prediction[:, 2], color="red")
plt.plot(chronos_prediction[:, 3], -chronos_prediction[:, 2], color="blue")

steps = np.arange(0, prediction_length, step)
axins = inset_axes(plt.gca(), width="40%", height="20%", loc="upper right", borderpad=1)
axins.plot(steps + step, pft_metrics["smape"], color="red")
axins.plot(steps + step, chronos_metrics["smape"], color="blue")
axins.set_ylabel("sMAPE")
axins.set_xlabel("Prediction Length")
axins.set_xticks([32, 64, 96, 128])
axins.set_xlim(step, prediction_length);

In [ ]:
# sanity check
fig, axes = plt.subplots(3, 1, figsize=(10, 4), sharex=True)
plt.subplots_adjust(hspace=0.0)

context_ts = np.arange(context_length + 1)
pred_ts = np.arange(context_length, context_length + prediction_length)
for i, ax in enumerate(axes.flatten()):
    ax.plot(
        context_ts,
        subsampled_pendulum_data[: context_length + 1, i],
        color="black",
        alpha=0.2,
    )
    ax.plot(
        pred_ts,
        subsampled_pendulum_data[
            context_length : context_length + prediction_length, i
        ],
        color="black",
        linestyle="--",
        alpha=0.5,
    )
    ax.plot(pred_ts, pft_prediction[:, i], color="red")
    ax.plot(pred_ts, chronos_prediction[:, i], color="blue")

plt.show()

In [ ]:
pft_mean_metrics, pft_std_metrics, pft_starts, pft_predictions = (
    compute_rollout_metrics(
        pft_model,
        subsampled_pendulum_data,
        context_length,
        prediction_length,
        step=8,
        num_windows=10,
        sliding_context=True,
        limit_prediction_length=False,
    )
)
chronos_mean_metrics, chronos_std_metrics, chronos_starts, chronos_predictions = (
    compute_rollout_metrics(
        chronos,
        subsampled_pendulum_data,
        context_length,
        prediction_length,
        step=8,
        num_windows=10,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=True,
        transpose=True,
    )
)

In [ ]:
# sanity check
fig, axes = plt.subplots(3, 1, figsize=(10, 4), sharex=True)
plt.subplots_adjust(hspace=0.0)

total_ts = np.arange(len(subsampled_pendulum_data))
for i, ax in enumerate(axes.flatten()):
    ax.plot(
        total_ts,
        subsampled_pendulum_data[:, i],
        color="black",
        alpha=0.2,
    )
    for j, (c, p) in enumerate(zip(chronos_starts, pft_starts)):
        pft_pred_ts = np.arange(
            p + context_length, p + context_length + prediction_length
        )
        chronos_context_ts = np.arange(c, c + context_length)
        chronos_pred_ts = np.arange(
            c + context_length, c + context_length + prediction_length
        )
        ax.plot(pft_pred_ts, pft_predictions[j][:, i], color="red", alpha=0.1)
        ax.plot(chronos_pred_ts, chronos_predictions[j][:, i], color="blue", alpha=0.1)

plt.show()


In [ ]:
plt.figure(figsize=(10, 4))
ts = np.arange(8, prediction_length + 8, 8)
plt.plot(ts, pft_mean_metrics["smape"], marker="o", color="red")
plt.fill_between(
    ts,
    pft_mean_metrics["smape"] - pft_std_metrics["smape"],
    pft_mean_metrics["smape"] + pft_std_metrics["smape"],
    color="red",
    alpha=0.2,
)
plt.plot(ts, chronos_mean_metrics["smape"], marker="o", color="blue")
plt.fill_between(
    ts,
    chronos_mean_metrics["smape"] - chronos_std_metrics["smape"],
    chronos_mean_metrics["smape"] + chronos_std_metrics["smape"],
    color="blue",
    alpha=0.2,
)
plt.xticks(ts)
plt.show()


# Eigenworms

In [ ]:
INDEX = 9
fpath = f"{base_dir}/worm_behavior/data/worm_{INDEX}.pkl"
worm_data = np.load(fpath, allow_pickle=True)[2048::1]
eigenworms = loadmat(f"{base_dir}/worm_behavior/data/EigenWorms.mat")["EigenWorms"]

# de-NaN the data with linear interpolation
time_idx = np.arange(len(worm_data))
for d in range(worm_data.shape[1]):
    mask = np.isnan(worm_data[:, d])
    if mask.any():
        valid = ~mask
        worm_data[:, d] = np.interp(time_idx, time_idx[valid], worm_data[valid, d])
assert not np.isnan(worm_data).any()

print(worm_data.shape)

### Make Video

In [ ]:
# from IPython.display import HTML
# from matplotlib.animation import FuncAnimation


# def reconstruct_worm(coeffs, eigenworms, segment_length=1.0):
#     """
#     Reconstruct a worm from its coefficients and the eigenworms.

#     Args:
#         coeffs: The coefficients of the worm (n_timesteps, n_eigenworms)
#         eigenworms: The eigenworms (n_features, n_eigenworms)
#         segment_length: The length of each segment of the worm.

#     Returns:
#         The reconstructed worm.
#     """
#     T, nworms = coeffs.shape
#     n_segments = eigenworms.shape[0]
#     basis = eigenworms[:, :nworms]
#     theta = coeffs @ basis.T

#     x = np.zeros((T, n_segments + 1))
#     y = np.zeros((T, n_segments + 1))
#     x[:, 1:] = segment_length * np.cos(theta)
#     y[:, 1:] = segment_length * np.sin(theta)

#     return x.cumsum(axis=1), y.cumsum(axis=1)


# def animate_worm(x, y, num_frames=200, interval=50, save_path=None):
#     """
#     Create an animation of the worm's movement over time.

#     Args:
#         x: Array of x coordinates with shape (T, n_segments+1)
#         y: Array of y coordinates with shape (T, n_segments+1)
#         num_frames: Number of frames to include in the animation
#         interval: Time between frames in milliseconds

#     Returns:
#         HTML animation that can be displayed in the notebook
#     """
#     fig, ax = plt.subplots(figsize=(8, 6))

#     # Set consistent axis limits for the animation
#     x_min, x_max = x.min(), x.max()
#     y_min, y_max = y.min(), y.max()

#     # Add some padding to the limits
#     x_padding = (x_max - x_min) * 0.1
#     y_padding = (y_max - y_min) * 0.1

#     ax.set_xlim(x_min - x_padding, x_max + x_padding)
#     ax.set_ylim(y_min - y_padding, y_max + y_padding)
#     ax.set_aspect("equal")
#     ax.set_title("Worm Movement")

#     # Create line and fill objects
#     line = ax.plot([], [], "b-", lw=2)[0]
#     fill = ax.fill([], [], color="blue")
#     time_text = ax.text(0.02, 0.95, "", transform=ax.transAxes)

#     # Calculate width profile - increases toward middle, decreases toward ends
#     n_points = x.shape[1]
#     width_profile = np.zeros(n_points)
#     max_width = 3  # Maximum width of the worm body
#     for i in range(n_points):
#         arg = 2 * i / (n_points - 1) - 1  # normalize to [-1, 1]
#         width_profile[i] = max_width * (
#             1
#             / (1 + np.exp(-8 * (arg + 0.7)))
#             * (1 - 1 / (1 + np.exp(-8 * (arg - 0.7))))
#         )

#     def init():
#         line.set_data([], [])
#         fill[0].set_xy(np.zeros((0, 2)))
#         time_text.set_text("")
#         return line, fill[0], time_text

#     def update(frame):
#         # Update the centerline
#         line.set_data(x[frame], y[frame])

#         # Calculate perpendicular vectors for width
#         dx = np.diff(x[frame])
#         dy = np.diff(y[frame])
#         # Normalize and rotate 90 degrees to get perpendicular direction
#         lengths = np.sqrt(dx**2 + dy**2)
#         nx = -dy / lengths
#         ny = dx / lengths

#         # Create polygon vertices for the worm body
#         vertices = []

#         # Top edge (add points from head to tail)
#         for i in range(n_points - 1):
#             vertices.append(
#                 (
#                     x[frame][i] + width_profile[i] * nx[i],
#                     y[frame][i] + width_profile[i] * ny[i],
#                 )
#             )

#         # Bottom edge (add points from tail to head)
#         for i in range(n_points - 2, -1, -1):
#             vertices.append(
#                 (
#                     x[frame][i] - width_profile[i] * nx[i],
#                     y[frame][i] - width_profile[i] * ny[i],
#                 )
#             )

#         # Update the fill
#         fill[0].set_xy(vertices)
#         time_text.set_text(f"Frame: {frame}")

#         return line, fill[0], time_text

#     # Use a subset of frames if there are too many
#     total_frames = min(num_frames, len(x))
#     frame_indices = np.linspace(0, len(x) - 1, total_frames, dtype=int)

#     anim = FuncAnimation(
#         fig, update, frames=frame_indices, init_func=init, blit=True, interval=interval
#     )
#     if save_path is not None:
#         anim.save(save_path, writer="ffmpeg")
#     plt.close()
#     return HTML(anim.to_jshtml())


# # Create and display the animation
# x, y = reconstruct_worm(worm_data, eigenworms)
# worm_animation = animate_worm(x[:1000], y[:1000], save_path="../figures/wormanim.mp4")

# worm_animation

### Forecast Worms

In [ ]:
context_length = 512
prediction_length = 128

In [ ]:
pft_prediction = forecast(
    pft_model,
    worm_data[:context_length],
    prediction_length,
    limit_prediction_length=False,
    sliding_context=True,
    differenced=True,
)

pft_metrics = defaultdict(list)
step = 8
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        pft_prediction[0 : i + step],
        worm_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        pft_metrics[metric].append(submetrics[metric])
print(pft_metrics)

In [ ]:
chronos_prediction = forecast(
    chronos,
    worm_data[:context_length],
    prediction_length,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    deterministic=True,
    differenced=True,
)

chronos_metrics = defaultdict(list)
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        chronos_prediction[0 : i + step],
        worm_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        chronos_metrics[metric].append(submetrics[metric])
print(chronos_metrics)

In [ ]:
plot_comparison(
    worm_data,
    pft_prediction,
    chronos_prediction,
    context_length,
    prediction_length,
    pft_metrics,
    chronos_metrics,
    step=step,
)

In [ ]:
# sanity check
_ = plot_model_prediction(
    pft_model,
    worm_data,
    context_length=context_length,
    prediction_length=prediction_length,
    sliding_context=True,
    limit_prediction_length=False,
    differenced=True,
)
_ = plot_model_prediction(
    chronos_ft,
    worm_data,
    context_length=context_length,
    prediction_length=prediction_length,
    transpose=True,
    num_samples=1,
    limit_prediction_length=False,
    deterministic=True,
    differenced=True,
    color="blue",
)

In [ ]:
pft_mean_metrics, pft_std_metrics, pft_starts, pft_predictions = (
    compute_rollout_metrics(
        pft_model,
        worm_data,
        context_length,
        prediction_length,
        step=8,
        num_windows=10,
        sliding_context=True,
        limit_prediction_length=False,
        differenced=True,
    )
)
chronos_mean_metrics, chronos_std_metrics, chronos_starts, chronos_predictions = (
    compute_rollout_metrics(
        chronos,
        worm_data,
        context_length,
        prediction_length,
        step=8,
        num_windows=10,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=True,
        differenced=True,
        transpose=True,
    )
)

In [ ]:
plt.figure(figsize=(10, 4))
ts = np.arange(8, prediction_length + 8, 8)
plt.plot(ts, pft_mean_metrics["smape"], marker="o", color="red")
plt.fill_between(
    ts,
    pft_mean_metrics["smape"] - pft_std_metrics["smape"],
    pft_mean_metrics["smape"] + pft_std_metrics["smape"],
    color="red",
    alpha=0.2,
)
plt.plot(ts, chronos_mean_metrics["smape"], marker="o", color="blue")
plt.fill_between(
    ts,
    chronos_mean_metrics["smape"] - chronos_std_metrics["smape"],
    chronos_mean_metrics["smape"] + chronos_std_metrics["smape"],
    color="blue",
    alpha=0.2,
)
plt.ylabel("sMAPE")
plt.xticks(ts)
plt.show()

# Turbulent Boundary Layer

In [ ]:
turbpca_data = np.load(
    f"{base_dir}/turbulence/BLexp_Re980_pca10.pkl", allow_pickle=True
)
subsampled_turbpca_data = turbpca_data[512::1]
print(subsampled_turbpca_data.shape)

In [ ]:
context_length = 512
prediction_length = 256

In [ ]:
pft_prediction = forecast(
    pft_model,
    subsampled_turbpca_data[:context_length],
    prediction_length,
    limit_prediction_length=False,
    sliding_context=True,
    differenced=True,
)

pft_metrics = defaultdict(list)
step = 8
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        pft_prediction[0 : i + step],
        turbpca_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        pft_metrics[metric].append(submetrics[metric])
print(pft_metrics)

In [ ]:
chronos_prediction = forecast(
    chronos,
    subsampled_turbpca_data[:context_length],
    prediction_length,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    deterministic=True,
    differenced=True,
)

chronos_metrics = defaultdict(list)
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        chronos_prediction[0 : i + step],
        turbpca_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        chronos_metrics[metric].append(submetrics[metric])
print(chronos_metrics)

In [ ]:
plot_comparison(
    turbpca_data,
    pft_prediction,
    chronos_prediction,
    context_length,
    prediction_length,
    pft_metrics,
    chronos_metrics,
    step=step,
)

In [ ]:
# sanity check
_ = plot_model_prediction(
    pft_model,
    subsampled_turbpca_data,
    context_length,
    prediction_length,
    sliding_context=True,
    limit_prediction_length=False,
    differenced=True,
)
_ = plot_model_prediction(
    chronos,
    subsampled_turbpca_data,
    context_length,
    prediction_length,
    transpose=True,
    num_samples=1,
    limit_prediction_length=False,
    deterministic=True,
    differenced=True,
    color="blue",
)

# Electronic Circuit

In [ ]:
netfpath = f"{base_dir}/electronic_circuit/Structure/Net_1.dat"
# fpath = f"{base_dir}/electronic_circuit/R1/ST_0_3.dat"
fpath = f"{base_dir}/electronic_circuit/R1/ST_50_3.dat"
net = np.loadtxt(netfpath)
circuit_data = np.loadtxt(fpath)
print(net.shape, circuit_data.shape)

In [ ]:
context_length = 512
prediction_length = 512

In [ ]:
pft_prediction = forecast(
    pft_model,
    circuit_data[:context_length],
    prediction_length,
    limit_prediction_length=False,
    sliding_context=True,
)

pft_metrics = defaultdict(list)
step = 64
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        pft_prediction[0 : i + step],
        circuit_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        pft_metrics[metric].append(submetrics[metric])
print(pft_metrics)

In [ ]:
chronos_ft_prediction = forecast(
    chronos_ft,
    circuit_data[:context_length],
    prediction_length,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    deterministic=True,
)

chronos_ft_metrics = defaultdict(list)
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        chronos_ft_prediction[0 : i + step],
        circuit_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        chronos_ft_metrics[metric].append(submetrics[metric])
print(chronos_ft_metrics)

In [ ]:
chronos_zs_prediction = forecast(
    chronos_zs,
    circuit_data[:context_length],
    prediction_length,
    transpose=True,
    limit_prediction_length=False,
    num_samples=1,
    deterministic=True,
)

chronos_zs_metrics = defaultdict(list)
for i in np.arange(0, prediction_length, step):
    submetrics = compute_metrics(
        chronos_zs_prediction[0 : i + step],
        circuit_data[context_length : context_length + i + step],
        include=metrics,
    )
    for metric in metrics:
        chronos_zs_metrics[metric].append(submetrics[metric])
print(chronos_zs_metrics)

In [ ]:
circuit_data.shape

In [ ]:
pft_prediction.shape

In [ ]:
plot_comparison(
    circuit_data,
    {
        "Our Model": pft_prediction,
        "Chronos 20M Finetune": chronos_ft_prediction,
        "Chronos 20M": chronos_zs_prediction,
    },
    context_length,
    prediction_length,
    legend_kwargs={"loc": "center right", "frameon": True},
    save_path="figs/circuit_comparison.pdf",
)

In [ ]:
circuit_data.shape

In [ ]:
start_time = 0

In [ ]:
# sanity check
_ = plot_model_prediction(
    pft_model,
    circuit_data[start_time:],
    512,
    512,
    sliding_context=True,
    limit_prediction_length=False,
)
_ = plot_model_prediction(
    chronos_zs,
    circuit_data[start_time:],
    512,
    512,
    transpose=True,
    num_samples=1,
    limit_prediction_length=False,
    color="blue",
)

### Coupling Strength Scaling Law

In [ ]:
# classify data by type {1, 2, 3} and sort by coupling strength
fpaths = os.listdir(f"{base_dir}/electronic_circuit/R1")
ec_fpaths = defaultdict(list)
for fpath in fpaths:
    ec_fpaths[int(fpath.split("_")[2][0])].append(fpath)
for k, v in ec_fpaths.items():
    ec_fpaths[k] = sorted(v, key=lambda x: int(x.split("_")[1]))
print(ec_fpaths)

In [ ]:
ec_fpaths[3][-1]

In [ ]:
n_steps = 8
pft_errors = {
    k: {m: np.zeros((n_steps, len(v))) for m in metrics} for k, v in ec_fpaths.items()
}
chronos_errors = {
    k: {m: np.zeros((n_steps, len(v))) for m in metrics} for k, v in ec_fpaths.items()
}
for k, v in tqdm(
    ec_fpaths.items(), desc="Processing type-k circuit data", total=len(ec_fpaths)
):
    for i, fpath in tqdm(
        enumerate(v), desc=f"Processing type-{k} circuit data", total=len(v)
    ):
        circuit_data = np.loadtxt(f"{base_dir}/electronic_circuit/R1/{fpath}")
        pft_prediction = forecast(
            pft_model,
            circuit_data[:context_length],
            prediction_length,
            limit_prediction_length=False,
            sliding_context=True,
        )
        chronos_prediction = forecast(
            chronos_ft,
            circuit_data[:context_length],
            prediction_length,
            transpose=True,
            limit_prediction_length=False,
            num_samples=1,
        )
        for chunk, j in enumerate(
            np.arange(0, prediction_length, prediction_length // n_steps)
        ):
            pft_submetrics = compute_metrics(
                pft_prediction[0 : j + step],
                circuit_data[context_length : context_length + j + step],
                include=metrics,
            )
            chronos_submetrics = compute_metrics(
                chronos_prediction[0 : j + step],
                circuit_data[context_length : context_length + j + step],
                include=metrics,
            )
            for metric in metrics:
                pft_errors[k][metric][chunk, i] = pft_submetrics[metric]
                chronos_errors[k][metric][chunk, i] = chronos_submetrics[metric]

In [ ]:
mean_pft_errors = {
    m: np.mean([pft_errors[k][m] for k in ec_fpaths], axis=0) for m in metrics
}
mean_chronos_errors = {
    m: np.mean([chronos_errors[k][m] for k in ec_fpaths], axis=0) for m in metrics
}

# std_pft_errors = {
#     m: np.std([pft_errors[k][m] for k in ec_fpaths], axis=0) for m in metrics
# }
# std_chronos_errors = {
#     m: np.std([chronos_errors[k][m] for k in ec_fpaths], axis=0) for m in metrics
# }
ste_pft_errors = {
    m: np.std([pft_errors[k][m] for k in ec_fpaths], axis=0) / np.sqrt(len(ec_fpaths))
    for m in metrics
}
ste_chronos_errors = {
    m: np.std([chronos_errors[k][m] for k in ec_fpaths], axis=0)
    / np.sqrt(len(ec_fpaths))
    for m in metrics
}

# rescale the smape
mean_pft_errors["smape"] = mean_pft_errors["smape"] / 2
mean_chronos_errors["smape"] = mean_chronos_errors["smape"] / 2
ste_pft_errors["smape"] = ste_pft_errors["smape"] / 2
ste_chronos_errors["smape"] = ste_chronos_errors["smape"] / 2

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(mean_pft_errors["smape"][-1], label="Our Model")
plt.plot(mean_chronos_errors["smape"][-1], label="Chronos 20M Finetune")
plt.fill_between(
    np.arange(len(mean_pft_errors["smape"][-1])),
    mean_pft_errors["smape"][-1] - ste_pft_errors["smape"][-1],
    mean_pft_errors["smape"][-1] + ste_pft_errors["smape"][-1],
    alpha=0.2,
)
plt.fill_between(
    np.arange(len(mean_chronos_errors["smape"][-1])),
    mean_chronos_errors["smape"][-1] - ste_chronos_errors["smape"][-1],
    mean_chronos_errors["smape"][-1] + ste_chronos_errors["smape"][-1],
    alpha=0.2,
)
plt.xlabel("Coupling Strength", fontweight="bold")
plt.ylabel("sMAPE", fontweight="bold")
plt.legend(loc="center right", frameon=True)
plt.tight_layout()
plt.savefig("figs/circuit_coupling_strength_scaling.pdf", bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.title(r"% $\Delta$sMAPE", fontweight="bold")
percentage_error = (
    chronos_errors[k]["smape"] - pft_errors[k]["smape"]
) / chronos_errors[k]["smape"]
plt.imshow(percentage_error, cmap="RdBu_r", label=f"Type-{k}", aspect="auto")
plt.ylabel("Prediction Length", fontweight="bold")
plt.yticks(
    np.arange(n_steps),
    np.arange(0, prediction_length, prediction_length // n_steps)
    + prediction_length // n_steps,
)
plt.xlabel("Coupling Strength", fontweight="bold")
plt.tight_layout()
plt.savefig(
    "figs/circuit_coupling_strength_scaling_percentage_error.pdf", bbox_inches="tight"
)
plt.show()

# ECG

In [ ]:
fpath = f"{base_dir}/electrocardiogram/ecg_train.csv.gz"
ecg_data = np.loadtxt(fpath, delimiter=",")
print(ecg_data.shape)

In [ ]:
context_length = 512
prediction_length = 512
start = 0
stride = 1

subsampled_ecg_data = ecg_data[start::stride]

In [ ]:
start_times_to_show = np.arange(0, len(subsampled_ecg_data) - 1024, 20000)
print(len(start_times_to_show))
print(start_times_to_show)

In [ ]:
start_times_extra = np.arange(4096, len(subsampled_ecg_data) - 1024, 4096)
print(len(start_times_extra))
print(start_times_extra)

In [ ]:
start_times = np.concatenate([start_times_to_show, start_times_extra])
print(len(start_times))

In [ ]:
compute_metrics_times = np.arange(64, prediction_length + 64, 64)
print(compute_metrics_times)

In [ ]:
metrics

In [ ]:
our_rollout_metrics = defaultdict(dict)
our_predictions = defaultdict()

for start_time in tqdm(
    start_times, desc=f"Forecasting PFT for {len(start_times)} context windows..."
):
    pft_prediction = forecast(
        pft_model,
        subsampled_ecg_data[start_time : start_time + context_length],
        prediction_length,
        limit_prediction_length=False,
        sliding_context=True,
    )
    our_predictions[start_time] = pft_prediction
    for t in compute_metrics_times:
        submetrics = compute_metrics(
            pft_prediction[0:t],
            subsampled_ecg_data[context_length : context_length + t],
            include=metrics,
        )
        for metric_name in metrics:
            if t not in our_rollout_metrics[metric_name]:
                our_rollout_metrics[metric_name][t] = []
            our_rollout_metrics[metric_name][t].append(submetrics[metric_name])
print(our_rollout_metrics["smape"])

In [ ]:
chronos_ft_rollout_metrics = defaultdict(dict)
chronos_ft_predictions = defaultdict()

for start_time in tqdm(
    start_times,
    desc=f"Forecasting Chronos FT for {len(start_times)} context windows...",
):
    chronos_prediction = forecast(
        chronos_ft,
        subsampled_ecg_data[start_time : start_time + context_length],
        prediction_length,
        transpose=True,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=True,
    )
    chronos_ft_predictions[start_time] = chronos_prediction
    for t in compute_metrics_times:
        submetrics = compute_metrics(
            chronos_prediction[0:t],
            subsampled_ecg_data[context_length : context_length + t],
            include=metrics,
        )
        for metric_name in metrics:
            if t not in chronos_ft_rollout_metrics[metric_name]:
                chronos_ft_rollout_metrics[metric_name][t] = []
            chronos_ft_rollout_metrics[metric_name][t].append(submetrics[metric_name])
print(chronos_ft_rollout_metrics["smape"])

In [ ]:
chronos_zs_rollout_metrics = defaultdict(dict)
chronos_zs_predictions = defaultdict()

for start_time in tqdm(
    start_times,
    desc=f"Forecasting Chronos ZS for {len(start_times)} context windows...",
):
    chronos_prediction = forecast(
        chronos_zs,
        subsampled_ecg_data[start_time : start_time + context_length],
        prediction_length,
        transpose=True,
        limit_prediction_length=False,
        num_samples=1,
        deterministic=True,
    )
    chronos_zs_predictions[start_time] = chronos_prediction
    for t in compute_metrics_times:
        submetrics = compute_metrics(
            chronos_prediction[0:t],
            subsampled_ecg_data[context_length : context_length + t],
            include=metrics,
        )
        for metric_name in metrics:
            if t not in chronos_zs_rollout_metrics[metric_name]:
                chronos_zs_rollout_metrics[metric_name][t] = []
            chronos_zs_rollout_metrics[metric_name][t].append(submetrics[metric_name])
print(chronos_zs_rollout_metrics["smape"])

In [ ]:
subsampled_ecg_data.shape

In [ ]:
baseline_mean_rollout_metrics = defaultdict(dict)
baseline_mean_predictions = defaultdict()

for start_time in tqdm(
    start_times, desc=f"Forecasting PFT for {len(start_times)} context windows..."
):
    baseline_mean_prediction = np.mean(
        np.expand_dims(
            subsampled_ecg_data[start_time : start_time + context_length], axis=0
        ),
        axis=0,
    )
    baseline_mean_predictions[start_time] = baseline_mean_prediction
    for t in compute_metrics_times:
        submetrics = compute_metrics(
            baseline_mean_prediction[0:t],
            subsampled_ecg_data[context_length : context_length + t],
            include=metrics,
        )
        for metric_name in metrics:
            if t not in baseline_mean_rollout_metrics[metric_name]:
                baseline_mean_rollout_metrics[metric_name][t] = []
            baseline_mean_rollout_metrics[metric_name][t].append(
                submetrics[metric_name]
            )
print(baseline_mean_rollout_metrics["smape"])

In [ ]:
# get default colors
colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]
print(colors)

In [ ]:
prediction_length_to_show = 512

# Plot predictions and metrics
context_ts = np.arange(context_length + 1)
pred_ts = np.arange(context_length, context_length + prediction_length_to_show)
error_ts = (
    np.arange(context_length - step, context_length + prediction_length_to_show, step)
    + step
)


selected_start_times = [4096, 40_000, 60_000]
n_examples = len(selected_start_times)

fig, axes = plt.subplots(n_examples, 1, figsize=(6, 3), sharex=True)
plt.subplots_adjust(hspace=0.0)

linewidth = 1
for i, start_time in enumerate(selected_start_times):
    print(start_time)
    axes[i].plot(
        context_ts,
        subsampled_ecg_data[start_time : start_time + context_length + 1],
        color="black",
        alpha=0.8,
        linewidth=linewidth,
        label="context",
    )
    axes[i].plot(
        pred_ts,
        subsampled_ecg_data[
            start_time + context_length : start_time
            + context_length
            + prediction_length_to_show
        ],
        color="black",
        linestyle="-",
        linewidth=linewidth,
        label="groundtruth",
        alpha=0.8,
    )
    axes[i].plot(
        pred_ts,
        our_predictions[start_time][:prediction_length_to_show],
        # color=colors[0],
        linewidth=linewidth,
        zorder=10,
        # alpha=0.8,
    )
    axes[i].plot(
        pred_ts,
        chronos_ft_predictions[start_time][:prediction_length_to_show],
        # color=colors[1],
        linewidth=linewidth,
        alpha=0.8,
        zorder=2,
    )
    axes[i].plot(
        pred_ts,
        chronos_zs_predictions[start_time][:prediction_length_to_show],
        # color=colors[2],
        linewidth=linewidth if i < 2 else 3,
        alpha=0.8 if i < 2 else 1,
        zorder=1 if i < 2 else 8,
    )

    axes[i].set_xticks([])
    axes[i].set_yticks([])

plt.tight_layout()
plt.savefig("figs/ecg_all_models_forecasts.pdf", bbox_inches="tight")
plt.show()

In [ ]:
tmp = np.array([1, 2, 3])
tmp = tmp / 2
tmp

In [ ]:
# Calculate mean, standard deviation, and standard error for each model
def calculate_stats(
    data_dict: dict[int, np.ndarray | list[float]], use_rescaled_smape: bool = False
):
    if use_rescaled_smape:
        data_dict = {t: list(np.array(v) / 2) for t, v in data_dict.items()}
    mean_vals = {t: np.mean(v) for t, v in data_dict.items()}
    median_vals = {t: np.median(v) for t, v in data_dict.items()}
    std_vals = {t: np.std(v) for t, v in data_dict.items()}
    ste_vals = {t: std_vals[t] / np.sqrt(len(data_dict[t])) for t in data_dict.keys()}
    return mean_vals, median_vals, std_vals, ste_vals


# Helper function to plot a model's results with error bands
def plot_model_results(
    mean_dict: dict[int, float],
    ste_dict: dict[int, float],
    marker: str,
    label: str,
    color: str | None = None,
):
    x_values = list(mean_dict.keys())
    y_values = list(mean_dict.values())
    y_errors = list(ste_dict.values())

    plt.plot(x_values, y_values, marker=marker, label=label, color=color)
    plt.fill_between(
        x_values,
        np.array(y_values) - np.array(y_errors),
        np.array(y_values) + np.array(y_errors),
        alpha=0.1,
    )


metric_name = "smape"
metric_name_title = "sMAPE"
# Calculate statistics for all models
mean_metric_ours, median_metric_ours, _, ste_metric_ours = calculate_stats(
    our_rollout_metrics[metric_name], use_rescaled_smape=metric_name == "smape"
)
mean_metric_chronos_ft, median_metric_chronos_ft, _, ste_metric_chronos_ft = (
    calculate_stats(
        chronos_ft_rollout_metrics[metric_name],
        use_rescaled_smape=metric_name == "smape",
    )
)
mean_metric_chronos_zs, median_metric_chronos_zs, _, ste_metric_chronos_zs = (
    calculate_stats(
        chronos_zs_rollout_metrics[metric_name],
        use_rescaled_smape=metric_name == "smape",
    )
)
mean_metric_baseline_mean, median_metric_baseline_mean, _, ste_metric_baseline_mean = (
    calculate_stats(
        baseline_mean_rollout_metrics[metric_name],
        use_rescaled_smape=metric_name == "smape",
    )
)

# Create the plot
plt.figure(figsize=(4, 3))

# Plot each model
plot_model_results(mean_metric_ours, ste_metric_ours, "o", label="Our Model")
plot_model_results(
    mean_metric_chronos_ft, ste_metric_chronos_ft, "s", label="Chronos 20M Finetune"
)
plot_model_results(
    mean_metric_chronos_zs, ste_metric_chronos_zs, "v", label="Chronos ZS"
)
plot_model_results(
    mean_metric_baseline_mean,
    ste_metric_baseline_mean,
    "D",
    label="Mean Baseline",
    color="gray",
)

# Add labels and show
plt.xlabel("Prediction Length", fontweight="bold")
plt.xticks(list(mean_metric_ours.keys()))
plt.title("ECG", fontweight="bold")
plt.ylabel(metric_name_title, fontweight="bold")
# plt.title(metric_name_title, fontweight="bold")
# Set y-axis to use scientific notation
# plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.ylim(13, None)

plt.legend(loc="lower center", frameon=True, fontsize=8, ncol=2)
plt.tight_layout()
plt.savefig(f"figs/ecg_all_models_{metric_name}.pdf", bbox_inches="tight")
plt.show()